In [1]:
import re
import sys
import pandas as pd
import numpy as np
from simple_io import IO
import codecs
from jieba import posseg
import time
import operator
import jieba

In [4]:
all_wxmsg=client.queryAsPandas(all_wxmsg)

Query ID = appuser_20180404153315_37775102-8b4c-40aa-91cb-61f853ad952c
Total jobs = 2
sql compiled.start to submit application.
Launching Job 1 out of 2
Running with YARN Application = application_1520329373387_280338
Kill Command = /home/hadoop/bin/yarn application -kill application_1520329373387_280338

RemoteSparkJobMonitor Query Hive on Spark job[0] stages: [0, 1, 5, 2, 6, 3, 4]

Status: Running (Hive on Spark job[0])
LocalSparkJobMonitor Job Progress Format
CurrentTime StageId_StageAttemptId: SucceededTasksCount(+RunningTasksCount-FailedTasksCount)/TotalTasksCount
2018-04-04 15:33:43,519	Stage-0_0: 0/3	Stage-1_0: 0/3	Stage-2_0: 0/2	Stage-3_0: 0/3	Stage-4_0: 0/3	Stage-5_0: 0/2	Stage-6_0: 0/2	
2018-04-04 15:33:44,527	Stage-0_0: 0(+1)/3	Stage-1_0: 0(+1)/3	Stage-2_0: 0/2	Stage-3_0: 0/3	Stage-4_0: 0/3	Stage-5_0: 0/2	Stage-6_0: 0/2	
2018-04-04 15:33:47,545	Stage-0_0: 0(+3)/3	Stage-1_0: 0(+1)/3	Stage-2_0: 0/2	Stage-3_0: 0/3	Stage-4_0: 0/3	Stage-5_0: 0/2	Stage-6_0: 0/2	
2018-04-04 15:33:4

In [5]:
after_mars_msg=client.queryAsPandas(after_mars_msg)

Query ID = appuser_20180404155609_fa2ad8ec-1636-43db-a721-ebdc903c6f71
Total jobs = 2
sql compiled.start to submit application.
Launching Job 1 out of 2
Running with YARN Application = application_1520329373387_280338
Kill Command = /home/hadoop/bin/yarn application -kill application_1520329373387_280338

RemoteSparkJobMonitor Query Hive on Spark job[2] stages: [8]

Status: Running (Hive on Spark job[2])
LocalSparkJobMonitor Job Progress Format
CurrentTime StageId_StageAttemptId: SucceededTasksCount(+RunningTasksCount-FailedTasksCount)/TotalTasksCount
2018-04-04 15:56:11,468	Stage-8_0: 0/3	
2018-04-04 15:56:14,473	Stage-8_0: 0/3	
2018-04-04 15:56:17,477	Stage-8_0: 0/3	
2018-04-04 15:56:18,479	Stage-8_0: 0(+3)/3	
2018-04-04 15:56:21,485	Stage-8_0: 0(+3)/3	
2018-04-04 15:56:24,493	Stage-8_0: 0(+3)/3	
2018-04-04 15:56:25,495	Stage-8_0: 1(+2)/3	
2018-04-04 15:56:26,497	Stage-8_0: 3/3 Finished	
Status: Finished successfully in 16.03 seconds
Launching Job 2 out of 2
Running with YARN Applica

In [6]:
transfert_ia_customer_wechat=pd.read_csv('transfer_customer_ia.csv')
#transfert_ia_customer_wechatid=transfert_ia_customer_wechat.as_matrix()
#transfert_ia_customer_wechatid=tuple([tuple(row) for row in transfert_ia_customer_wechatid])

In [7]:
transfert_ia_customer_wechat.describe

<bound method NDFrame.describe of                     ia_id          customer_id          submit_time
0     wxid_9nis050pbwcv22   wxid_1895058950615  2017-04-25 11:30:37
1     wxid_7ldghy19v40t22            caile1985  2017-12-22 12:30:43
2     wxid_6sm0rrow7o2222  wxid_1w8y1zow5mib21  2017-06-22 17:08:02
3     wxid_bog38vb8wbox22  wxid_lzurod7h589411  2017-01-26 16:30:46
4     wxid_bih2ir75304k22  wxid_q3mmarq8j94721  2018-03-29 16:26:37
5     wxid_lqvya0d2ovgw22            sdfindnxb  2017-12-13 17:13:49
6     wxid_ftm41p6aia4h22         huangwei5507  2017-04-10 14:36:35
7     wxid_9aovm3o4lmqf22            nicolas72  2016-12-05 10:01:32
8     wxid_231zv6behgxh22  wxid_ytaqam7gf45q11  2017-09-27 10:55:33
9     wxid_bog38vb8wbox22  wxid_fok2csezvy8421  2017-05-25 10:28:00
10    wxid_tvj3l2nky5il22       fancystar-king  2017-08-16 17:30:15
11    wxid_bog38vb8wbox22  wxid_j2qs2b8c5eso21  2017-11-23 16:37:12
12    wxid_dicw3d6x8s7f22   wxid_2789707896511  2017-12-14 10:19:50
13    wxid_lqv

In [8]:
transfert_cutomer_wechat=pd.read_csv('transfer_wechat.csv')
transfert_cutomer_wechatid=transfert_cutomer_wechat.as_matrix().flatten()

In [10]:
after_mars_msg=after_mars_msg.drop_duplicates(['t1.uuid'])
#all_wxmsg=all_wxmsg.drop_duplicates(['wechat_msg.uuid'])

In [11]:
all_wxmsg = all_wxmsg.rename(columns={'wechat_msg.iawechatid': 'ia_id','wechat_msg.userwechatid': 'customer_id'})

In [12]:
result = pd.merge(all_wxmsg, transfert_ia_customer_wechat, on=['ia_id', 'customer_id'])

In [14]:
grouped_transfert_msg=result.groupby(['ia_id', 'customer_id'])

In [14]:
print(len(grouped_transfert_msg))

1780


In [151]:
def distrubted(data):
    content_time_pd=pd.DataFrame(columns = ['wechat_msg_distinct.iawechatid','wechat_msg_distinct.userwechatid',"wechat_msg_distinct.time""wechat_msg_distinct.content", "wechat_msg_distinct.createtime"])
    #first_time=[]
    msgs_content=[]
    ia_id=[]
    user_id=[]
    distribution=[]
    time=[]
    for groupname, groupdata in data:
        groupdata=groupdata.sort_values(by='wechat_msg.createtime')
        groupdata['wechat_msg.createtime']=pd.to_datetime(groupdata['wechat_msg.createtime'])
        groupdata['submit_time']=pd.to_datetime(groupdata['submit_time'])
        valid_groupdata=groupdata[groupdata['wechat_msg.createtime']<groupdata['submit_time']]
        if (len(valid_groupdata)<50):
            continue
        #print(max(groupdata['submit_time']-groupdata['wechat_msg.createtime']).days)
        ia_id.append(groupname[0])
        user_id.append(groupname[1])
        time.append(str(max(groupdata['submit_time']-groupdata['wechat_msg.createtime']).days))
        distribution.append(len(valid_groupdata))
        msg_content=valid_groupdata['wechat_msg.content'].as_matrix().flatten()
        #first_chat=valid_groupdata['wechat_msg_distinct.createtime'].as_matrix().flatten()[0]
        msgs_content.append(msg_content)
        #first_time.append(first_chat)
    
    content_time_pd['wechat_msg_distinct.content']=msgs_content
    content_time_pd['wechat_msg_distinct.time']=time
    content_time_pd['wechat_msg_distinct.iawechatid']=ia_id
    content_time_pd['wechat_msg_distinct.userwechatid']=user_id
    return content_time_pd,distribution

In [152]:
content_time_pd,distribution=distrubted(grouped_transfert_msg)
#content_time_pd.to_csv("positive_dataframe.csv")

In [153]:
print(len(content_time_pd))

1566


In [25]:
content_time_pd.to_csv("positive_dataframe.csv")

In [6]:
content_time_pd=pd.read_csv("positive_dataframe.csv")

In [42]:
distribution=np.array(distribution)
median_positive=np.median(distribution)
print(median_positive)
print(np.average(distribution))
print(min(distribution))

184.0
266.9247005134056
1


In [43]:
len(content_time_pd)

1753

In [28]:
untransfer_msg=after_mars_msg[~after_mars_msg['t1.userwechatid'].isin(transfert_ia_customer_wechat['customer_id'])]

In [29]:
untransfer_msg=untransfer_msg.groupby(['t1.iawechatid','t1.userwechatid'])

In [67]:
from datetime import datetime
def sample_selection(data):
    #content_time_pd=pd.DataFrame(columns = ['wechat_msg_distinct.iawechatid','wechat_msg_distinct.userwechatid',"wechat_msg_distinct.time""wechat_msg_distinct.content", "wechat_msg_distinct.createtime"])
    total_msg=[]
    labels=[]
    now= datetime.now()
    time=[]
    for groupname, groupdata in data:
        groupdata=groupdata.sort_values(by='t1.createtime')
        groupdata['t1.createtime']=pd.to_datetime(groupdata['t1.createtime'])
        msg_content=groupdata['t1.content'].as_matrix().flatten()
        #print(msg_content)
        if len(msg_content)<10:
            continue
        labels.append(0)
        long_msg_content=""
        time.append(str(max(now-groupdata['t1.createtime']).days))
        for i in range(0,len(msg_content)):
            if i<2:
                continue
            long_msg_content+=str(msg_content[i])
        ##print(long_msg_content)
        #print(type(long_msg_content))
        #break
        total_msg.append(long_msg_content)
    #total_msg=np.array(total_msg)
    #labels=np.array(labels)
    return total_msg,labels,time

In [68]:
X,y,time=sample_selection(untransfer_msg)

In [44]:
print(len(X))

15045


In [45]:
content_time_pd=content_time_pd.groupby(['wechat_msg_distinct.iawechatid','wechat_msg_distinct.userwechatid'])

In [72]:
def sample_positive(data):
    labels=[]
    total_msg=[]
    time=[]
    for groupname, groupdata in data:
        labels.append(1)
        #print(type(item))
        time.append(str(max(groupdata['wechat_msg_distinct.time'])))
        time_content=str(groupdata['wechat_msg_distinct.content'].as_matrix().flatten())
        total_msg.append(time_content)
    return total_msg,labels,time

In [149]:
new_X,new_y,transfert_time=sample_positive(content_time_pd)

In [74]:
print(new_y.count(1))
print(len(new_X))
print(len(new_y))

1753
1753
1753


In [75]:
sample_X=new_X+X
sample_y=new_y+y

In [86]:
sample_time=transfert_time+time

In [76]:
print(len(sample_X))

16798


In [51]:
def creadstoplist(stopwordspath):
    stwlist = [line.strip() for line in codecs.open(stopwordspath, 'r', encoding='utf-8').readlines()]
    tmp = range(len(stwlist))
    return dict(zip(stwlist,tmp))

In [52]:
stwdic = creadstoplist('stopwords.txt')
jieba.load_userdict("mydict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.823 seconds.
Prefix dict has been built succesfully.


In [53]:
def token_line(line):
    seqlist =jieba.posseg.cut(line) # True False
    flags=filter(lambda w: w.word not in stwdic.keys() and len(w.word.strip())>0, seqlist)
    corpus=[w.word for w in flags]
    str_word = ' '.join(corpus).replace(",","")
    str_flags = ' '.join(["{}/{}".format(i.word,i.flag) for i in flags])
    #print('line length:{:d},cost time:{:f}'.format(len(corpus),time.time()-start))
    return str_flags,str_word

In [54]:
def remove_punctuation(line):
    rule = re.compile("[^\u4e00-\u9fa5^.^a-z^A-Z^0-9]")
    line = rule.sub('',line)
    return line

In [77]:
def div_word_all_words(total_content):
    msg_feature=[]
    for content in total_content:
        content=remove_punctuation(content)
        str_flags,str_word=token_line(content)
        words=str_word.split()
        sentence_feature=[]
        for word in words:
            sentence_feature.append(word)
        msg_feature.append(sentence_feature)
    #msg_feature=np.array(msg_feature)
    return msg_feature

In [78]:
div_word_X=div_word_all_words(sample_X)

## method 1

In [79]:
import pickle
from gensim.models.word2vec import Word2Vec

In [80]:
w2v_builder = Word2Vec.load('w2v_model.pkl')

In [81]:
len_arr=[]
for article in div_word_X:
    length_article=len(article)
    len_arr.append(length_article)

In [82]:
len_arr=np.asarray(len_arr)
median=np.median(len_arr)
print(median)

159.0


In [83]:
MAX_SEQUENCE_LENGTH = int(median) # 每条聊天记录最大长度 coming from k
EMBEDDING_DIM = 300

In [18]:
def buildWordVector(text, MAX_SEQUENCE_LENGTH,dimension, w2v_builder):
    vec = np.zeros(dimension*MAX_SEQUENCE_LENGTH).reshape((MAX_SEQUENCE_LENGTH, dimension))
    #count = 0.
    i=0
    for word in text:
        try:
            i+=1
            if (i==len(vec)):
                break
            vec[i]=(w2v_builder[word].reshape((dimension)))
            #count += 1.
        except KeyError:
            continue
    return vec

In [19]:
numerical_train_X=[]
for item in div_word_X:
    vec=buildWordVector(item,MAX_SEQUENCE_LENGTH,EMBEDDING_DIM, w2v_builder)
    numerical_train_X.append(vec)
#numerical_train_X=np.array(numerical_train_X)

/home/hub/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [80]:
from sklearn.cross_validation  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(numerical_train_X, sample_y, test_size=0.2, random_state=42)

In [81]:
X_train=X_train+(numerical_train_X[:1000])

In [82]:
y_train=y_train+(sample_y[:1000])

In [83]:
X_train=np.array(X_train)

In [84]:
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [85]:
print(X_train.shape)
print(y_train.shape)

(14438, 160, 300)
(14438,)


In [86]:
X_train, X_val, y_train, Y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [140]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

model = Sequential()
model.add(Conv1D(filters=250,kernel_size=3,input_shape=(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
#plot_model(model, to_file='model.png',show_shapes=True)

/home/wangshuopeng/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 157, 250)          225250    
_________________________________________________________________
dropout_1 (Dropout)          (None, 157, 250)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 52, 250)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 13000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               3900300   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
Total params: 4,125,851
Trainable params: 4,125,851
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=64)
model.save('word_vector_cnn.h')
#print(model.evaluate(X_test, y_test))

Train on 11550 samples, validate on 2888 samples
Epoch 1/10
11550/11550 [==============================] - 67s - loss: 2.6988 - acc: 0.8306 - val_loss: 2.5232 - val_acc: 0.8431
Epoch 2/10
11550/11550 [==============================] - 72s - loss: 2.7104 - acc: 0.8318 - val_loss: 2.5229 - val_acc: 0.8435
Epoch 3/10
11550/11550 [==============================] - 67s - loss: 2.7106 - acc: 0.8316 - val_loss: 2.5218 - val_acc: 0.8431
Epoch 4/10
11550/11550 [==============================] - 66s - loss: 2.7103 - acc: 0.8317 - val_loss: 2.5227 - val_acc: 0.8435
Epoch 5/10
11550/11550 [==============================] - 66s - loss: 2.7102 - acc: 0.8318 - val_loss: 2.5224 - val_acc: 0.8435
Epoch 6/10
11550/11550 [==============================] - 65s - loss: 2.7103 - acc: 0.8317 - val_loss: 2.5229 - val_acc: 0.8431
Epoch 7/10
11550/11550 [==============================] - 64s - loss: 2.7102 - acc: 0.8318 - val_loss: 2.5220 - val_acc: 0.8435
Epoch 8/10
11550/11550 [==============================]

In [61]:
from sklearn.metrics import cohen_kappa_score
out = model.predict_classes(X_test)
out = np.array(out)

cohen_kappa_score(y_test,out)

3360/3360 [==============================] - 5s     


0.0

In [90]:
from keras.layers import LSTM
model_LSTM = Sequential()
model_LSTM.add(LSTM(output_dim=50,input_shape=(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM)))
model_LSTM.add(Dropout(0.2))
model_LSTM.add(Dense(EMBEDDING_DIM, activation='relu'))
model_LSTM.add(Dense(1, activation='sigmoid'))
model_LSTM.summary()

/home/hub/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(160, 300), units=50)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 50)                70200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 300)               15300     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 301       
Total params: 85,801
Trainable params: 85,801
Non-trainable params: 0
_________________________________________________________________


In [91]:
model_LSTM.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model_LSTM.fit(X_train, y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=64)
model_LSTM.save('word_vector_lstm.h')

Train on 11550 samples, validate on 2888 samples
Epoch 1/10
11550/11550 [==============================] - 41s - loss: 0.3726 - acc: 0.8521 - val_loss: 0.3040 - val_acc: 0.8798
Epoch 2/10
11550/11550 [==============================] - 39s - loss: 0.2904 - acc: 0.8887 - val_loss: 0.2800 - val_acc: 0.8961
Epoch 3/10
11550/11550 [==============================] - 39s - loss: 0.2622 - acc: 0.9028 - val_loss: 0.2643 - val_acc: 0.9082
Epoch 4/10
11550/11550 [==============================] - 40s - loss: 0.3381 - acc: 0.8516 - val_loss: 0.3121 - val_acc: 0.8722
Epoch 5/10
11550/11550 [==============================] - 39s - loss: 0.2773 - acc: 0.8928 - val_loss: 0.3088 - val_acc: 0.8708
Epoch 6/10
11550/11550 [==============================] - 39s - loss: 0.2606 - acc: 0.8952 - val_loss: 0.3061 - val_acc: 0.8722
Epoch 7/10
11550/11550 [==============================] - 39s - loss: 0.2312 - acc: 0.9133 - val_loss: 0.2321 - val_acc: 0.9148
Epoch 8/10
11550/11550 [==============================]

In [92]:
out = model_LSTM.predict_classes(X_test)
out = np.array(out)

cohen_kappa_score(y_test,out)

3360/3360 [==============================] - 4s     


0.7615899599152133

## 方案二，对于每个句子的特征，我们通过sentence vector的形式来表达。依然是用中位数长度来选择看的词数，但是通过累加各个特征值，并除以总得词数来表示一个句子。对于样本不均衡的问题，用smote的方法来增加正例样本数目

In [1]:
import re
import sys
import pandas as pd
import numpy as np
from simple_io import IO
import codecs
from jieba import posseg
import time
import operator
import jieba

In [2]:
import pickle
from gensim.models.word2vec import Word2Vec
w2v_builder = Word2Vec.load('w2v_model.pkl')

In [3]:
corpus=pd.read_csv("corpus.csv")

In [4]:
well_words=corpus.values

In [5]:
well_words=well_words.tolist()

In [7]:
def word_ratio(flags):
    #print ' '.join(sample)
    if type(flags) is str:
        flags = flags.split(' ')
    else :
        pass
    
    corpus = well_words
    
    dictionary = {}
    count = 0
    if flags is np.nan or flags is None:
        return 0,-1
    else :
        for key in corpus:
            dictionary[key[0]] = 0
        for word in flags:
            if word in dictionary.keys():
                dictionary[word] += 1
        for v in dictionary.values():
            count += v
        try :
            ratio = float(count)/float(len(flags))
        except ZeroDivisionError:
            return 0,-1
    return len(flags), ratio

In [6]:
all_features=pd.read_csv("Information/feature_content.csv")

In [8]:
all_features=all_features.drop(['flags','flags.1','flags.2','add_time','end_time','cus_time_first','ia_time_first'], axis=1)

In [9]:
all_features = pd.concat([all_features,all_features['content'].apply(lambda x: pd.Series(word_ratio(x)))],axis = 1)
all_features.rename(columns={0:'text_all_seg_num',1:'high_freq_ratio'},inplace=True)

In [10]:
all_features = pd.concat([all_features,all_features['ia_content'].apply(lambda x: pd.Series(word_ratio(x)))],axis = 1)
all_features.rename(columns={0:'ia_text_seg_num',1:'ia_high_freq_ratio'},inplace=True)
all_features = pd.concat([all_features,all_features['cus_content'].apply(lambda x: pd.Series(word_ratio(x)))],axis = 1)
all_features.rename(columns={0:'cus_text_seg_num',1:'cus_high_freq_ratio'},inplace=True)

In [11]:
MAX_SEQUENCE_LENGTH=int((all_features['ia_text_num'].sum()+all_features['cus_text_num'].sum())/len(all_features))
EMBEDDING_DIM=300

In [12]:
postive=all_features[all_features['label']==1]
negative=all_features[all_features['label']==0]
negative=negative[:10000]

In [13]:
all_features=pd.concat([postive,negative])

In [14]:
print(MAX_SEQUENCE_LENGTH)

178


In [15]:
all_features=all_features.dropna(axis=0, how='any')

In [16]:
len(all_features)

11499

In [17]:
div_word_X=all_features['content'].as_matrix()

In [19]:
def sentence_vec(text,MAX_SEQUENCE_LENGTH,EMBEDDING_DIM, w2v_builder):
    vec = np.zeros(EMBEDDING_DIM).reshape((1,EMBEDDING_DIM))
    count = 0.
    sentence=text.split(" ")
    for word in sentence:
        try:
            vec += w2v_builder[word].reshape((1,EMBEDDING_DIM))
            count += 1.
            if count>MAX_SEQUENCE_LENGTH:
                break
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [20]:
setence_vec=[]
i=0
for item in div_word_X:
    vec=sentence_vec(item,MAX_SEQUENCE_LENGTH,EMBEDDING_DIM, w2v_builder)
    setence_vec.append(vec)

/home/hub/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [21]:
setence_vec=np.concatenate(setence_vec)

In [20]:
setence_vec=np.concatenate(numerical_train_X)

In [28]:
setence_vec.shape

(2046822, 300)

In [21]:
df = all_features.join(pd.DataFrame(setence_vec)).reset_index(drop=True)

In [22]:
df=df.dropna(axis=0, how='any')

In [23]:
len(df)

11499

In [24]:
y=df['label'].as_matrix().flatten()
df=df.drop(['label','content','ia_content','cus_content','ia_id','customer_wx_id'],axis=1)

In [25]:
X=df.as_matrix()

# the last item is the sentence vetector

In [26]:
from sklearn.cross_validation  import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

/home/wangshuopeng/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [28]:
(test_vec)=X_test[:,-300:]
train_vec=X_train[:,-300:]

In [29]:
test_vec=test_vec.reshape((len(X_test),EMBEDDING_DIM,1))

In [30]:
X_train, X_val, y_train, Y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [31]:
train_vec=X_train[:,-300:]
val_vec=X_val[:,-300:]

In [32]:
train_vec=train_vec.reshape((len(X_train),EMBEDDING_DIM,1))
val_vec=val_vec.reshape((len(X_val),EMBEDDING_DIM,1))

In [33]:
from keras.layers import Dense, Input, Flatten, SpatialDropout1D,SpatialDropout2D
from keras.layers import Conv1D, MaxPooling1D, Embedding,BatchNormalization
from keras.models import Sequential
setence_model = Sequential()
setence_model.add(Conv1D(filters=250,kernel_size=3,input_shape=(EMBEDDING_DIM,1)))
setence_model.add(BatchNormalization())
setence_model.add(SpatialDropout1D(0.2))
setence_model.add(MaxPooling1D(4))
setence_model.add(Flatten())
setence_model.add(Dense(EMBEDDING_DIM, activation='relu'))
setence_model.add(Dense(1, activation='sigmoid'))
setence_model.summary()

/home/wangshuopeng/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 298, 250)          1000      
_________________________________________________________________
batch_normalization_1 (Batch (None, 298, 250)          1000      
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 298, 250)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 74, 250)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18500)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               5550300   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
Total para

In [34]:
class_weight = {0 : 1.,1: 10.,}
setence_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
setence_model.fit(train_vec, y_train, validation_data=(val_vec, Y_val), epochs=5, batch_size=128,shuffle=True, class_weight = class_weight)
setence_model.save('sentence_vector_cnn.h')
#print(setence_model.evaluate(X_test, y_test))

Train on 7248 samples, validate on 1812 samples
Epoch 1/5
7248/7248 [==============================] - 39s - loss: 23.0973 - acc: 0.8413 - val_loss: 1.1524 - val_acc: 0.8560
Epoch 2/5
7248/7248 [==============================] - 38s - loss: 23.5723 - acc: 0.8538 - val_loss: 1.5868 - val_acc: 0.8560
Epoch 3/5
7248/7248 [==============================] - 38s - loss: 23.5723 - acc: 0.8538 - val_loss: 2.0942 - val_acc: 0.8560
Epoch 4/5
7248/7248 [==============================] - 38s - loss: 23.5723 - acc: 0.8538 - val_loss: 2.3160 - val_acc: 0.8560
Epoch 5/5
7248/7248 [==============================] - 42s - loss: 23.5723 - acc: 0.8538 - val_loss: 2.3216 - val_acc: 0.8560


In [36]:
new_train_feature=setence_model.predict_proba(train_vec)
print(new_train_feature)

7248/7248 [==============================] - 11s    
[[9.8332827e-14]
 [1.7929767e-12]
 [1.5080224e-11]
 ...
 [3.5911416e-13]
 [3.7783166e-12]
 [6.9649770e-09]]


In [38]:
new_test_feature=setence_model.predict_proba(test_vec)
print(new_test_feature)

2240/2265 [============================>.] - ETA: 0s[[1.0569648e-14]
 [2.5920806e-12]
 [1.6774478e-14]
 ...
 [3.0299840e-10]
 [1.6315036e-11]
 [2.2406033e-12]]


In [41]:
train_vec=train_vec.reshape((len(X_train),EMBEDDING_DIM))
test_vec=test_vec.reshape((len(X_test),EMBEDDING_DIM))

In [61]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc_model=rfc.fit(train_vec, y_train)

In [63]:
rfc_train_feature=rfc_model.predict_proba(train_vec)[:,1]
rfc_test_feature=rfc_model.predict_proba(test_vec)[:,1]

In [64]:
X_rfc_train=np.column_stack((X_train_nf,rfc_train_feature))
X_rfc_test=np.column_stack((X_test_nf,rfc_test_feature))

In [65]:
rfc_gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_rfc_train, y_train)
rfc_proba = rfc_gbm.predict_proba(X_rfc_test)[:,1]
y_rfc_pre=rfc_gbm.predict(X_rfc_test)
up =  heapq.nlargest(int(0.2*len(rfc_proba)), range(len(rfc_proba)), rfc_proba.__getitem__)
recall_score(y_test[up], y_rfc_pre[up])  

/home/wangshuopeng/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6888888888888889

In [42]:
from sklearn.svm import SVC
clf = SVC(probability=True)
svm_model = clf.fit(train_vec, y_train)

In [43]:
new_train_feature=svm_model.predict_proba(train_vec)[:,1]

In [45]:
new_test_feature=svm_model.predict_proba(test_vec)[:,1]

In [46]:
X_train_nf=X_train[:,:27]

In [47]:
X_train=np.column_stack((X_train_nf,new_train_feature))

In [48]:
X_test_nf=X_test[:,:27]
X_test=np.column_stack((X_test_nf,new_test_feature))

In [49]:
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)
predictions = gbm.predict_proba(X_test)

In [50]:
predict_proba=predictions[:,1]

In [51]:
y_pre=gbm.predict(X_test)

/home/wangshuopeng/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [52]:
import heapq

In [55]:
indexs =  heapq.nlargest(int(0.2*len(predict_proba)), range(len(predict_proba)), predict_proba.__getitem__)

In [56]:
from sklearn.metrics import recall_score
recall_score(y_test[indexs], y_pre[indexs])  

0.7218045112781954

In [57]:
gbm_1 = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train_nf, y_train)
without_pro = gbm_1.predict_proba(X_test_nf)[:,1]

In [58]:
without_y_pre=gbm_1.predict(X_test_nf)

/home/wangshuopeng/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [59]:
top =  heapq.nlargest(int(0.2*len(without_pro)), range(len(without_pro)), without_pro.__getitem__)

In [60]:
recall_score(y_test[top], without_y_pre[top])  

0.673469387755102